In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from operator import itemgetter, attrgetter
import time,datetime
from sklearn.cross_validation import train_test_split

import numexpr as ne
ne.set_num_threads(100000000)


def get_week_day(d):
    date=d.split('-')
    year=int (date[0])
    month=int(date[1])
    day=int(date[2])
    
    days=[31,28,31,30,31,30,31,31,30,31,30,31]
    if (year%4==0 and  year%100!=0) or (year%400==0):
        days[1]=29
    else:
        days[1]=28
    sum_day=0 
    for i in range(0,month-1):
        sum_day+=days[i]
    sum_day+=day
    return (year-1+int((year-1)/4)-int((year-1)/100)+int((year-1)/400)+sum_day)%7


def get_time(time):
    t1=time.split(',')
    t2=t1[0]
    t3=t2.split(' ')
    t4=t3[1]
    t5=t4.split(':')
    t6=int(t5[0])
    if t6<6:
        return 'Morning'
    elif t6<20:
        return 'Afternoon'
    else:
        return 'Evening'
    
    
def get_year(time):
    t1=time.split(',')
    t2=t1[0]
    t3=t2.split(' ')
    t4=t3[0]
    t5=t4.split('-')
    return int(t5[0][1:])

def get_month(time):
    t1=time.split(',')
    t2=t1[0]
    t3=t2.split(' ')
    t4=t3[0]
    t5=t4.split('-')
    return int(t5[1])

def get_day(time):
    t1=time.split(',')
    t2=t1[0]
    t3=t2.split(' ')
    t4=t3[0]
    t5=t4.split('-')
    return int(t5[2])

def get_hour(time):
    t1=time.split(',')
    t2=t1[0]
    t3=t2.split(' ')
    t4=t3[1]
    t5=t4.split(':')
    return int(t5[0])
def get_time_begin(time):
    t1=time.split(',')
    t2=t1[0]
    t3=t2.split(' ')
    t4=t3[1]
    t5=t4.split(':')
    return int(t5[1])

def get_time_end(time):
    t1=time.split(',')
    t2=t1[1]
    t3=t2.split(' ')
    t4=t3[1]
    t5=t4.split(':')
    return int(t5[1])

def get_time2(time):
    t1=time.split(',')
    t2=t1[0]
    t3=t2.split(' ')
    return t3[1]

def get_seconds(time):
    s=time.split(':')
    return int((int(s[0])*60+int(s[1]))*60)


def get_isHolidays(date):
    if date=='2016-04-02' or date=='2016-04-03' or date=='2016-04-04' or date=='2016-04-30' or date=='2016-05-01' or date=='2016-05-02' or date=='2016-06-09' or date=='2016-06-10' or date=='2016-06-11':
        return 1
    else:
        return 0
    



df1=pd.read_table('gy_contest_link_info.txt', header=0,sep=';', encoding='utf-8')
df2=pd.read_table('gy_contest_link_top.txt', header=0,sep=';', encoding='utf-8')
df3=pd.read_table('gy_contest_link_traveltime_training_data.txt', header=0,sep=';', encoding='utf-8')


df2=df2.fillna(0)
df2['inN']=0
df2['outN']=0
df2['inLengthAverage']=0.0
df2['inWidthAverage']=0.0
df2['outLengthAverage']=0.0
df2['outWidthAverage']=0.0


for i in range(len(df2)):
    inlink=df2.iat[i,1]
    outlink=df2.iat[i,2]
    if inlink==0:
        df2.iat[i,3]=0
        df2.iat[i,5]=df2.iat[i,6]=0.0    
    else:
        df2.iat[i,3]=len(inlink.split("#"))
        sum1=0.0
        sum2=0.0
        sum3=0.0
        sum4=0.0
        for k in range(0,len(inlink.split("#"))):
            sum1+=pow(df1[df1['link_ID']==inlink.split("#")[k]]['length'].mean(),2)
            sum2+=df1[df1['link_ID']==inlink.split("#")[k]]['length'].mean()
            sum3+=pow(df1[df1['link_ID']==inlink.split("#")[k]]['width'].mean(),2)
            sum4+=df1[df1['link_ID']==inlink.split("#")[k]]['width'].mean()
        df2.iat[i,5]=sum1/sum2
        df2.iat[i,6]=sum3/sum4
    if outlink==0:
        df2.iat[i,4]=0
        df2.iat[i,7]=df2.iat[i,8]=0.0  
    else:
        df2.iat[i,4]=len(outlink.split("#"))
        sum1=0.0
        sum2=0.0
        sum3=0.0
        sum4=0.0
        for j in range(0,len(outlink.split("#"))):
            sum1+=pow(df1[df1['link_ID']==outlink.split("#")[j]]['length'].mean(),2)
            sum2+=df1[df1['link_ID']==outlink.split("#")[j]]['length'].mean()
            sum3+=pow(df1[df1['link_ID']==outlink.split("#")[j]]['width'].mean(),2)
            sum4+=df1[df1['link_ID']==outlink.split("#")[j]]['width'].mean()
        df2.iat[i,7]=sum1/sum2
        df2.iat[i,8]=sum3/sum4
        
        
df4=pd.merge(df1,df2)
df5=pd.merge(df4,df3)


df5['week']=df5['date'].apply(lambda x:get_week_day(x))
df5['time_level']=df5['time_interval'].apply(lambda x:get_time(x))
df5['time']=df5['time_interval'].apply(lambda x:get_time2(x))
df5['isHolidays']=df5['date'].apply(lambda x:get_isHolidays(x))
df5['year']=df5['time_interval'].apply(lambda x:get_year(x))
df5['month']=df5['time_interval'].apply(lambda x:get_month(x))
df5['day']=df5['time_interval'].apply(lambda x:get_day(x))
df5['hour']=df5['time_interval'].apply(lambda x:get_hour(x))
df5['time_begin']=df5['time_interval'].apply(lambda x:get_time_begin(x))
df5['time_end']=df5['time_interval'].apply(lambda x:get_time_end(x))
df5['seconds']=df5['time'].apply(lambda x:get_seconds(x))


df_1=df5[df5['week']==1]
df_2=df5[df5['week']==2]
df_3=df5[df5['week']==3]
df_4=df5[df5['week']==4]
df_5=df5[df5['week']==5]
df_6=df5[df5['week']==6]
df_0=df5[df5['week']==0]



df_1.to_csv('1_n.txt',sep=';')
df_2.to_csv('2_n.txt',sep=';')
df_3.to_csv('3_n.txt',sep=';')
df_4.to_csv('4_n.txt',sep=';')
df_5.to_csv('5_n.txt',sep=';')
df_6.to_csv('6_n.txt',sep=';')
df_0.to_csv('0_n.txt',sep=';')

df5.head(10)


C:\ProgramData\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


,link_ID,length,width,link_class,in_links,out_links,inN,outN,inLengthAverage,inWidthAverage,...,time_level,time,isHolidays,year,month,day,hour,time_begin,time_end,seconds
0,4377906289869500514,57,3,1,4377906285525800514,4377906281969500514,1,1,43.0,12.0,...,Afternoon,12:08:00,0,2016,5,21,12,8,10,43680
1,4377906289869500514,57,3,1,4377906285525800514,4377906281969500514,1,1,43.0,12.0,...,Morning,02:56:00,0,2016,5,21,2,56,58,10560
2,4377906289869500514,57,3,1,4377906285525800514,4377906281969500514,1,1,43.0,12.0,...,Afternoon,08:56:00,0,2016,5,21,8,56,58,32160
3,4377906289869500514,57,3,1,4377906285525800514,4377906281969500514,1,1,43.0,12.0,...,Evening,23:12:00,0,2016,5,21,23,12,14,83520
4,4377906289869500514,57,3,1,4377906285525800514,4377906281969500514,1,1,43.0,12.0,...,Evening,22:46:00,0,2016,5,21,22,46,48,81960
5,4377906289869500514,57,3,1,4377906285525800514,4377906281969500514,1,1,43.0,12.0,...,Afternoon,09:36:00,0,2016,5,21,9,36,38,34560
6,4377906289869500514,57,3,1,4377906285525800514,4377906281969500514,1,1,43.0,12.0,...,Afternoon,15:00:00,0,2016,5,21,15,0,2,54000
7,4377906289869500514,57,3,1,4377906285525800514,4377906281969500514,1,1,43.0,12.0,...,Afternoon,17:50:00,0,2016,5,21,17,50,52,64200
8,4377906289869500514,57,3,1,4377906285525800514,4377906281969500514,1,1,43.0,12.0,...,Afternoon,14:48:00,0,2016,5,21,14,48,50,53280
9,4377906289869500514,57,3,1,4377906285525800514,4377906281969500514,1,1,43.0,12.0,...,Afternoon,14:18:00,0,2016,5,21,14,18,20,51480
